## Imports

In [11]:
import json
from shapely.geometry import shape, Point
import geopy.distance
import pprint

### Check if point inside polygon

In [12]:
# load GeoJSON file containing sectors
with open("Datos Abiertos - Bogota/Ordenamiento_Territorial - Sector_Catastral/Raw-data-sector-catastral-april-2023.geojson") as f:
    js = json.load(f)

# construct point based on lon/lat returned by geocoder

point = Point(-74.066837, 4.732316)

# check each polygon to see if it contains the point
for feature in js["features"]:
    polygon = shape(feature["geometry"])
    if polygon.contains(point):
        print("Found containing polygon:", feature["properties"])

Found containing polygon: {'SCACODIGO': '009131', 'SCATIPO': 0, 'SCANOMBRE': 'SANTA HELENA', 'SHAPE_Leng': 0.0349063726429, 'SHAPE_Area': 4.68564938336e-05}


### Get distance between points

In [ ]:
# load JSON file containing points
with open("Datos Abiertos - Bogota/Vivienda_Ciudad_Territorio - Ocupacion-Ilegal/GeoJson-Nov-2022.json") as f:
    js = json.load(f)

home = (4.732316, -74.066837)

# results = [x for x in js["features"] if geopy.distance.geodesic(home, (x["lat"], x["lon"])).m]
results = []
for feature in js["features"]:
    point = (feature["lat"], feature["lon"])
    if geopy.distance.geodesic(home, point).m <= 3000:
        feature["distance_m"] = geopy.distance.geodesic(home, point).m
        results.append(feature)

newlist = sorted(results, key=lambda d: d["distance_m"])
newlist = newlist[0:5]
display(newlist)

### Get polygons in radius

In [ ]:
# Raw data should be processed with QGIS before (to WGS84) in order to have Type and Coordinates inside geometry
with open("Datos Abiertos - Bogota/Ordenamiento_Territorial - Parques POT/data-parques-feb-2023-processed.geojson") as f:
    js = json.load(f)

point = Point(-74.066837, 4.732316)

results = []
for feature in js["features"]:
    poly = shape(feature["geometry"])
    feature["dist2poly1"] = poly.distance(point) * 111194.93  # Degress to M considering earth
    feature["dist2poly2"] = poly.boundary.distance(point) * 111194.93  # Degress to M considering earth
    if feature["dist2poly1"] < 2000:
        feature.pop("geometry", None)
        results.append(feature)

newlist = sorted(results, key=lambda d: d["dist2poly1"])
newlist = newlist[0:50]
pprint.pprint(newlist)
